# PyDough TPC-H

This notebook provides PyDough translations for 19 out of the 22 TPC-H benchmark queries. For each query we will reproduce the business context, borrowing from the defintion provided in the specification, the corresponding SQL query, and finally a valid PyDough implementation. The underlying schema of this data matches this example image from [TPC Benchmark H Standard Specification](https://www.tpc.org/tpc_documents_current_versions/pdf/tpc-h_v2.17.1.pdf).

![TPC-H schema from the Specification Document as of December 12, 2024](../images/tpc_h_schema.png)

In several places we have update the names in our metadata to be more human readable. It should also be noted that the TPC-H benchmark does not necessarily constitute the "simplest" way to express the SQL for each of these statements, but this comparison is still useful for understanding how to construct PyDough statements.

In addition, since the demo consists of running the PyDough statements on SQLite, an OLTP database, some of these statements are significantly slower to complete than they would be if run instead on an OLAP database.

In [ ]:
%load_ext pydough_jupyter_extensions

In [ ]:
import pydough
import datetime
import pandas as pd

In [ ]:
# Setup demo metadata
pydough.active_session.load_metadata_graph("../metadata/tpch_demo_graph.json", "TPCH");
pydough.active_session.connect_database("sqlite", database="../tpch.db");
# Avoid scientific notation
pd.options.display.float_format = '{:.6f}'.format

## Query 1

This query seeks to determine **the amount of business that was billed, shipped, and returned across**.

Here is the corresponding SQL:

```SQL
select
	l_returnflag,
	l_linestatus,
	sum(l_quantity) as sum_qty,
	sum(l_extendedprice) as sum_base_price,
	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
	avg(l_quantity) as avg_qty,
	avg(l_extendedprice) as avg_price,
	avg(l_discount) as avg_disc,
	count(*) as count_order
from
	lineitem
where
	l_shipdate <= date '1998-12-01' - interval '90' day
group by
	l_returnflag,
	l_linestatus
order by
	l_returnflag,
	l_linestatus
```

In [ ]:
%%pydough

selected_lines = lines.WHERE((ship_date <= datetime.date(1998, 9, 2)))
output = PARTITION(selected_lines, name="l", by=(return_flag, status))(
    L_RETURNFLAG=return_flag,
    L_LINESTATUS=status,
    SUM_QTY=SUM(l.quantity),
    SUM_BASE_PRICE=SUM(l.extended_price),
    SUM_DISC_PRICE=SUM(l.extended_price * (1 - l.discount)),
    SUM_CHARGE=SUM(l.extended_price * (1 - l.discount) * (1 + l.tax)),
    AVG_QTY=AVG(l.quantity),
    AVG_PRICE=AVG(l.extended_price),
    AVG_DISC=AVG(l.discount),
    COUNT_ORDER=COUNT(l),
).ORDER_BY(L_RETURNFLAG.ASC(), L_LINESTATUS.ASC())
pydough.to_df(output)

## Query 2

This question seeks to determine **which supplier should be selected to place an order for a given part in a given region**.

Here is the corresponding SQL:

```SQL
SELECT
    S_ACCTBAL,
    S_NAME,
    N_NAME,
    P_PARTKEY,
    P_MFGR,
    S_ADDRESS,
    S_PHONE,
    S_COMMENT
FROM
    PART,
    SUPPLIER,
    PARTSUPP,
    NATION,
    REGION
WHERE
    P_PARTKEY = PS_PARTKEY
    AND S_SUPPKEY = PS_SUPPKEY
    AND P_SIZE = 15
    AND P_TYPE LIKE '%BRASS'
    AND S_NATIONKEY = N_NATIONKEY
    AND N_REGIONKEY = R_REGIONKEY
    AND R_NAME = 'EUROPE'
    AND PS_SUPPLYCOST = (
        SELECT MIN(PS_SUPPLYCOST)
        FROM PARTSUPP, SUPPLIER, NATION, REGION
        WHERE P_PARTKEY = PS_PARTKEY
          AND S_SUPPKEY = PS_SUPPKEY
          AND S_NATIONKEY = N_NATIONKEY
          AND N_REGIONKEY = R_REGIONKEY
          AND R_NAME = 'EUROPE'
    )
ORDER BY
    S_ACCTBAL DESC,
    N_NAME,
    S_NAME,
    P_PARTKEY
LIMIT 100;
```

Notice the use of a correlated subqueries to determine the `PS_SUPPLYCOST` criteria. In PyDough this is handled naturally by simply first peforming the partition, applying the filter, and then navigating back to `selected_parts` in question, which avoids fully stepping out of the query.

In [ ]:
%%pydough

selected_parts = (
    nations.WHERE(region.name == "EUROPE")
    .suppliers.supply_records.part(
        s_acctbal=BACK(2).account_balance,
        s_name=BACK(2).name,
        n_name=BACK(3).name,
        s_address=BACK(2).address,
        s_phone=BACK(2).phone,
        s_comment=BACK(2).comment,
        supplycost=BACK(1).supplycost,
    )
    .WHERE(ENDSWITH(part_type, "BRASS") & (size == 15))
)
output = PARTITION(selected_parts, name="p", by=key)(
    best_cost=MIN(p.supplycost)
).p.WHERE(
    (supplycost == BACK(1).best_cost)
    & ENDSWITH(part_type, "BRASS")
    & (size == 15)
)(
    S_ACCTBAL=s_acctbal,
    S_NAME=s_name,
    N_NAME=n_name,
    P_PARTKEY=key,
    P_MFGR=manufacturer,
    S_ADDRESS=s_address,
    S_PHONE=s_phone,
    S_COMMENT=s_comment,
).TOP_K(
    100,
    by=(S_ACCTBAL.DESC(), N_NAME.ASC(), S_NAME.ASC(), P_PARTKEY.ASC()),
)
pydough.to_df(output)

## Query 3

The question seeks to find **the 10 unshipped orders with the highest value**.

Here is the corresponding SQL:

```SQL
SELECT
    L_ORDERKEY,
    SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS REVENUE,
    O_ORDERDATE,
    O_SHIPPRIORITY
FROM
    CUSTOMER,
    ORDERS,
    LINEITEM
WHERE
    C_MKTSEGMENT = 'BUILDING'
    AND C_CUSTKEY = O_CUSTKEY
    AND L_ORDERKEY = O_ORDERKEY
    AND O_ORDERDATE < DATE '1995-03-15'
    AND L_SHIPDATE > DATE '1995-03-15'
GROUP BY
    L_ORDERKEY,
    O_ORDERDATE,
    O_SHIPPRIORITY
ORDER BY
    REVENUE DESC,
    O_ORDERDATE
LIMIT 10;
```

In [ ]:
%%pydough

selected_lines = orders.WHERE(
    (customer.mktsegment == "BUILDING") & (order_date < datetime.date(1995, 3, 15))
).lines.WHERE(ship_date > datetime.date(1995, 3, 15))(
    BACK(1).order_date,
    BACK(1).ship_priority,
)
output = PARTITION(
    selected_lines, name="l", by=(order_key, order_date, ship_priority)
)(
    L_ORDERKEY=order_key,
    REVENUE=SUM(l.extended_price * (1 - l.discount)),
    O_ORDERDATE=order_date,
    O_SHIPPRIORITY=ship_priority,
).TOP_K(10, by=(REVENUE.DESC(), O_ORDERDATE.ASC(), L_ORDERKEY.ASC()))
pydough.to_df(output)

## Query 4

This question seeks to answer **how well the order priority system is working at ensuring that orders are delivered on time within a particular quarter**.

Here is the corresponding SQL:

```SQL
SELECT
    O_ORDERPRIORITY,
    COUNT(*) AS ORDER_COUNT
FROM
    ORDERS
WHERE
    O_ORDERDATE >= DATE '1993-07-01'
    AND O_ORDERDATE < DATE '1993-10-01'
    AND EXISTS (
        SELECT 1
        FROM LINEITEM
        WHERE L_ORDERKEY = O_ORDERKEY
          AND L_COMMITDATE < L_RECEIPTDATE
    )
GROUP BY
    O_ORDERPRIORITY
ORDER BY
    O_ORDERPRIORITY;
```

PyDough handles the exist functionality through its use of `HAS`, relying on the underlying relationship between collects to simplify the join/table logic.

In [ ]:
%%pydough

selected_lines = lines.WHERE(commit_date < receipt_date)
selected_orders = orders.WHERE(
    (order_date >= datetime.date(1993, 7, 1))
    & (order_date < datetime.date(1993, 10, 1))
    & HAS(selected_lines)
)
output = PARTITION(selected_orders, name="o", by=order_priority)(
    O_ORDERPRIORITY=order_priority,
    ORDER_COUNT=COUNT(o),
).ORDER_BY(O_ORDERPRIORITY.ASC())
pydough.to_df(output)

## Query 6

This question formulated a sample "what-if" question, asking **how was revenue impacted over a previous year due to the presence of certain discounts** and potentially speculating on future revenue due to a change in that discount. 

Here is the corresponding SQL:

```SQL
SELECT
    SUM(L_EXTENDEDPRICE * L_DISCOUNT) AS REVENUE
FROM
    LINEITEM
WHERE
    L_SHIPDATE >= DATE '1994-01-01'
    AND L_SHIPDATE < DATE '1995-01-01'
    AND L_DISCOUNT BETWEEN 0.05 AND 0.07
    AND L_QUANTITY < 24;
```

In [ ]:
%%pydough

selected_lines = lines.WHERE(
    (ship_date >= datetime.date(1994, 1, 1))
    & (ship_date < datetime.date(1995, 1, 1))
    & (0.05 <= discount)
    & (discount <= 0.07)
    & (quantity < 24)
)(amt=extended_price * discount)
output = TPCH(REVENUE=SUM(selected_lines.amt))
pydough.to_df(output)

## Query 7

This question asks about **the value of goods shipped between certain nations within a particular year** as a representation for future contract negotiations.

Here is the corresponding SQL:

```SQL
SELECT
    SUPP_NATION,
    CUST_NATION,
    L_YEAR,
    SUM(VOLUME) AS REVENUE
FROM (
    SELECT
        N1.N_NAME AS SUPP_NATION,
        N2.N_NAME AS CUST_NATION,
        EXTRACT(YEAR FROM L_SHIPDATE) AS L_YEAR,
        L_EXTENDEDPRICE * (1 - L_DISCOUNT) AS VOLUME
    FROM
        SUPPLIER,
        LINEITEM,
        ORDERS,
        CUSTOMER,
        NATION N1,
        NATION N2
    WHERE
        S_SUPPKEY = L_SUPPKEY
        AND O_ORDERKEY = L_ORDERKEY
        AND C_CUSTKEY = O_CUSTKEY
        AND S_NATIONKEY = N1.N_NATIONKEY
        AND C_NATIONKEY = N2.N_NATIONKEY
        AND (
            (N1.N_NAME = 'FRANCE' AND N2.N_NAME = 'GERMANY')
            OR (N1.N_NAME = 'GERMANY' AND N2.N_NAME = 'FRANCE')
        )
        AND L_SHIPDATE BETWEEN DATE '1995-01-01' AND DATE '1996-12-31'
) SHIPPING
GROUP BY
    SUPP_NATION,
    CUST_NATION,
    L_YEAR
ORDER BY
    SUPP_NATION,
    CUST_NATION,
    L_YEAR;
```

In [ ]:
%%pydough

line_info = lines(
    supp_nation=supplier.nation.name,
    cust_nation=order.customer.nation.name,
    l_year=YEAR(ship_date),
    volume=extended_price * (1 - discount),
).WHERE(
    (ship_date >= datetime.date(1995, 1, 1))
    & (ship_date <= datetime.date(1996, 12, 31))
    & (
        ((supp_nation == "FRANCE") & (cust_nation == "GERMANY"))
        | ((supp_nation == "GERMANY") & (cust_nation == "FRANCE"))
    )
)

output = PARTITION(line_info, name="l", by=(supp_nation, cust_nation, l_year))(
    SUPP_NATION=supp_nation,
    CUST_NATION=cust_nation,
    L_YEAR=l_year,
    REVENUE=SUM(l.volume),
).ORDER_BY(
    SUPP_NATION.ASC(),
    CUST_NATION.ASC(),
    L_YEAR.ASC(),
)
pydough.to_df(output)

## Query 8

This question asks about **the change in a nation's marketshare for a particular over a two year period**.

Here is the corresponding SQL:

```SQL
SELECT
    O_YEAR,
    SUM(CASE
            WHEN NATION = 'BRAZIL' THEN VOLUME
            ELSE 0
        END) / SUM(VOLUME) AS MKT_SHARE
FROM (
    SELECT
        EXTRACT(YEAR FROM O_ORDERDATE) AS O_YEAR,
        L_EXTENDEDPRICE * (1 - L_DISCOUNT) AS VOLUME,
        N2.N_NAME AS NATION
    FROM
        PART,
        SUPPLIER,
        LINEITEM,
        ORDERS,
        CUSTOMER,
        NATION N1,
        NATION N2,
        REGION
    WHERE
        P_PARTKEY = L_PARTKEY
        AND S_SUPPKEY = L_SUPPKEY
        AND L_ORDERKEY = O_ORDERKEY
        AND O_CUSTKEY = C_CUSTKEY
        AND C_NATIONKEY = N1.N_NATIONKEY
        AND N1.N_REGIONKEY = R_REGIONKEY
        AND R_NAME = 'AMERICA'
        AND S_NATIONKEY = N2.N_NATIONKEY
        AND O_ORDERDATE BETWEEN DATE '1995-01-01' AND DATE '1996-12-31'
        AND P_TYPE = 'ECONOMY ANODIZED STEEL'
) ALL_NATIONS
GROUP BY
    O_YEAR
ORDER BY
    O_YEAR;
```

In [ ]:
%%pydough

volume_data = (
    nations.suppliers.supply_records.WHERE(
        part.part_type == "ECONOMY ANODIZED STEEL"
    )
    .lines(volume=extended_price * (1 - discount))
    .order(
        o_year=YEAR(order_date),
        volume=BACK(1).volume,
        brazil_volume=IFF(BACK(4).name == "BRAZIL", BACK(1).volume, 0),
    )
    .WHERE(
        (order_date >= datetime.date(1995, 1, 1))
        & (order_date <= datetime.date(1996, 12, 31))
        & (customer.nation.region.name == "AMERICA")
    )
)
output = PARTITION(volume_data, name="v", by=o_year)(
    O_YEAR=o_year,
    MKT_SHARE=SUM(v.brazil_volume) / SUM(v.volume),
)
pydough.to_df(output)

## Query 9

This question asks about **the profit made for a particular line of parts looking at the supplying nation and particular year in question**.

Here is the corresponding SQL:

```SQL
SELECT
    NATION,
    O_YEAR,
    SUM(AMOUNT) AS SUM_PROFIT
FROM (
    SELECT
        N_NAME AS NATION,
        EXTRACT(YEAR FROM O_ORDERDATE) AS O_YEAR,
        L_EXTENDEDPRICE * (1 - L_DISCOUNT) - PS_SUPPLYCOST * L_QUANTITY AS AMOUNT
    FROM
        PART,
        SUPPLIER,
        LINEITEM,
        PARTSUPP,
        ORDERS,
        NATION
    WHERE
        S_SUPPKEY = L_SUPPKEY
        AND PS_SUPPKEY = L_SUPPKEY
        AND PS_PARTKEY = L_PARTKEY
        AND P_PARTKEY = L_PARTKEY
        AND O_ORDERKEY = L_ORDERKEY
        AND S_NATIONKEY = N_NATIONKEY
        AND P_NAME LIKE '%green%'
) PROFIT
GROUP BY
    NATION,
    O_YEAR
ORDER BY
    NATION,
    O_YEAR;
```

In [ ]:
%%pydough

selected_lines = nations.suppliers.supply_records.WHERE(
    CONTAINS(part.name, "green")
).lines(
    nation=BACK(3).name,
    o_year=YEAR(order.order_date),
    value=extended_price * (1 - discount) - BACK(1).supplycost * quantity,
)

output = PARTITION(selected_lines, name="l", by=(nation, o_year))(
    NATION=nation, O_YEAR=o_year, AMOUNT=SUM(l.value)
).ORDER_BY(NATION.ASC(), O_YEAR.DESC())
pydough.to_df(output)

## Query 10

This question **determines the top 20 customers based on impact in revenue due to having returned parts within a particular quarter**.

Here is the corresponding SQL:

```SQL
SELECT
    C_CUSTKEY,
    C_NAME,
    SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS REVENUE,
    C_ACCTBAL,
    N_NAME,
    C_ADDRESS,
    C_PHONE,
    C_COMMENT
FROM
    CUSTOMER,
    ORDERS,
    LINEITEM,
    NATION
WHERE
    C_CUSTKEY = O_CUSTKEY
    AND L_ORDERKEY = O_ORDERKEY
    AND O_ORDERDATE >= DATE '1993-10-01'
    AND O_ORDERDATE < DATE '1994-01-01'
    AND L_RETURNFLAG = 'R'
    AND C_NATIONKEY = N_NATIONKEY
GROUP BY
    C_CUSTKEY,
    C_NAME,
    C_ACCTBAL,
    C_PHONE,
    N_NAME,
    C_ADDRESS,
    C_COMMENT
ORDER BY
    REVENUE DESC
LIMIT 20;
```

In [ ]:
%%pydough

selected_lines = orders.WHERE(
    (order_date >= datetime.date(1993, 10, 1))
    & (order_date < datetime.date(1994, 1, 1))
).lines.WHERE(return_flag == "R")(amt=extended_price * (1 - discount))

output = customers(
    C_CUSTKEY=key,
    C_NAME=name,
    REVENUE=SUM(selected_lines.amt),
    C_ACCTBAL=acctbal,
    N_NAME=nation.name,
    C_ADDRESS=address,
    C_PHONE=phone,
    C_COMMENT=comment,
).TOP_K(20, by=(REVENUE.DESC(), C_CUSTKEY.ASC()))
pydough.to_df(output)

## Query 11

The question seeks **to identify the most import subset of available part supply value from a particular nation**.

Here is the corresponding SQL:

```SQL
SELECT
    PS_PARTKEY,
    SUM(PS_SUPPLYCOST * PS_AVAILQTY) AS VALUE
FROM
    PARTSUPP,
    SUPPLIER,
    NATION
WHERE
    PS_SUPPKEY = S_SUPPKEY
    AND S_NATIONKEY = N_NATIONKEY
    AND N_NAME = 'GERMANY'
GROUP BY
    PS_PARTKEY
HAVING
    SUM(PS_SUPPLYCOST * PS_AVAILQTY) > (
        SELECT
            SUM(PS_SUPPLYCOST * PS_AVAILQTY) * 0.0001
        FROM
            PARTSUPP,
            SUPPLIER,
            NATION
        WHERE
            PS_SUPPKEY = S_SUPPKEY
            AND S_NATIONKEY = N_NATIONKEY
            AND N_NAME = 'GERMANY'
    )
ORDER BY
    VALUE DESC;
```

Notice that the SQL query requires a correlated subquery that leverages having to compute the corresponding SUM. PyDough uses a single filter to simplify this logic and ensure that through use of `BACK` the original results can be filtered.

In [ ]:
%%pydough
is_german_supplier = supplier.nation.name == "GERMANY"
selected_records = supply_records.WHERE(is_german_supplier)(metric=supplycost * availqty)
output = TPCH(min_market_share=SUM(selected_records.metric) * 0.0001).PARTITION(
    selected_records, name="ps", by=part_key
)(
    PS_PARTKEY=part_key, VALUE=SUM(ps.metric)
).WHERE(VALUE > BACK(1).min_market_share).ORDER_BY(VALUE.DESC())
pydough.to_df(output)

## Query 12

This question seeks to determine **the impact of low cost shipping modes on delays for orders of various priorities**.

Here is the corresponding SQL:

```SQL
SELECT
    L_SHIPMODE,
    SUM(CASE
            WHEN O_ORDERPRIORITY = '1-URGENT'
              OR O_ORDERPRIORITY = '2-HIGH'
            THEN 1
            ELSE 0
        END) AS HIGH_LINE_COUNT,
    SUM(CASE
            WHEN O_ORDERPRIORITY <> '1-URGENT'
              AND O_ORDERPRIORITY <> '2-HIGH'
            THEN 1
            ELSE 0
        END) AS LOW_LINE_COUNT
FROM
    ORDERS,
    LINEITEM
WHERE
    O_ORDERKEY = L_ORDERKEY
    AND L_SHIPMODE IN ('MAIL', 'SHIP')
    AND L_COMMITDATE < L_RECEIPTDATE
    AND L_SHIPDATE < L_COMMITDATE
    AND L_RECEIPTDATE >= DATE '1994-01-01'
    AND L_RECEIPTDATE < DATE '1995-01-01'
GROUP BY
    L_SHIPMODE
ORDER BY
    L_SHIPMODE;
```

In [ ]:
%%pydough

selected_lines = lines.WHERE(
    ((ship_mode == "MAIL") | (ship_mode == "SHIP"))
    & (ship_date < commit_date)
    & (commit_date < receipt_date)
    & (receipt_date >= datetime.date(1994, 1, 1))
    & (receipt_date < datetime.date(1995, 1, 1))
)(
    is_high_priority=(order.order_priority == "1-URGENT")
    | (order.order_priority == "2-HIGH"),
)
output = PARTITION(selected_lines, "l", by=ship_mode)(
    L_SHIPMODE=ship_mode,
    HIGH_LINE_COUNT=SUM(l.is_high_priority),
    LOW_LINE_COUNT=SUM(~(l.is_high_priority)),
).ORDER_BY(L_SHIPMODE.ASC())
pydough.to_df(output)

## Query 13

This question seeks to **understand the order count of each customer, including any customer who has not placed an order**.

Here is the corresponding SQL:

```SQL
SELECT
    C_COUNT,
    COUNT(*) AS CUSTDIST
FROM (
    SELECT
        C_CUSTKEY,
        COUNT(O_ORDERKEY) AS C_COUNT
    FROM
        CUSTOMER
        LEFT OUTER JOIN ORDERS ON C_CUSTKEY = O_CUSTKEY
        AND O_COMMENT NOT LIKE '%special%requests%'
    GROUP BY
        C_CUSTKEY
) C_ORDERS
GROUP BY
    C_COUNT
ORDER BY
    CUSTDIST DESC,
    C_COUNT DESC;
```

In [ ]:
%%pydough

customer_info = customers(
    key,
    num_non_special_orders=COUNT(
        orders.WHERE(~(LIKE(comment, "%special%requests%")))
    ),
)
output = PARTITION(customer_info, name="custs", by=num_non_special_orders)(
    C_COUNT=num_non_special_orders, CUSTDIST=COUNT(custs)
).ORDER_BY(CUSTDIST.DESC(), C_COUNT.DESC())
pydough.to_df(output)

## Query 14

This question seeks seeks to determine **the impact of a promotion on revenue within a particular month**.

Here is the corresponding SQL:

```SQL
SELECT
    100.00 * SUM(CASE
                     WHEN P_TYPE LIKE 'PROMO%'
                     THEN L_EXTENDEDPRICE * (1 - L_DISCOUNT)
                     ELSE 0
                 END) / SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS PROMO_REVENUE
FROM
    LINEITEM,
    PART
WHERE
    L_PARTKEY = P_PARTKEY
    AND L_SHIPDATE >= DATE '1995-09-01'
    AND L_SHIPDATE < DATE '1995-10-01';
```

In [ ]:
%%pydough

value = extended_price * (1 - discount)
selected_lines = lines.WHERE(
    (ship_date >= datetime.date(1995, 9, 1))
    & (ship_date < datetime.date(1995, 10, 1))
)(
    value=value,
    promo_value=IFF(STARTSWITH(part.part_type, "PROMO"), value, 0),
)
output = TPCH(PROMO_REVENUE=100.0 * SUM(selected_lines.promo_value) / SUM(selected_lines.value))
pydough.to_df(output)

## Query 15

This question seeks to determine **the top supplier within a particular quarter**.

Here is the corresponding SQL:

```SQL
with REVENUE0 AS
    SELECT
        L_SUPPKEY,
        SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT))
    FROM
        LINEITEM
    WHERE
        L_SHIPDATE >= DATE '1996-01-01'
        AND L_SHIPDATE < DATE '1996-04-01'
    GROUP BY
        L_SUPPKEY;

SELECT
    SUPPLIER_NO,
    TOTAL_REVENUE
FROM
    REVENUE0
WHERE
    TOTAL_REVENUE = (
        SELECT
            MAX(TOTAL_REVENUE)
        FROM
            REVENUE0
    )
ORDER BY
    SUPPLIER_NO;
```

In [ ]:
%%pydough

selected_lines = lines.WHERE(
    (ship_date >= datetime.date(1996, 1, 1))
    & (ship_date < datetime.date(1996, 4, 1))
)
total = SUM(selected_lines.extended_price * (1 - selected_lines.discount))
output = TPCH(
    max_revenue=MAX(suppliers(total_revenue=total).total_revenue)
).suppliers(
    S_SUPPKEY=key,
    S_NAME=name,
    S_ADDRESS=address,
    S_PHONE=phone,
    TOTAL_REVENUE=total,
).WHERE(TOTAL_REVENUE == BACK(1).max_revenue).ORDER_BY(S_SUPPKEY.ASC())
pydough.to_df(output)

## Query 16

This question seeks to determine **how many suppliers can meet the needs of a part with a particular set of attributes**.

Here is the corresponding SQL:

```SQL
select
    p_brand,
    p_type,
    p_size,
    count(distinct ps_suppkey) as supplier_cnt
from
    partsupp,
    part
where
    p_partkey = ps_partkey
    and p_brand <> 'BRAND#45'
    and p_type not like 'MEDIUM POLISHED%'
    and p_size in (49, 14, 23, 45, 19, 3, 36, 9)
    and ps_suppkey not in (
        select
            s_suppkey
        from
            supplier
        where
            s_comment like '%Customer%Complaints%'
    )
group by
    p_brand,
    p_type,
    p_size
order by
    supplier_cnt desc,
    p_brand,
    p_type,
    p_size;
```

In [ ]:
%%pydough

selected_records = (
    parts.WHERE(
        (brand != "BRAND#45")
        & ~STARTSWITH(part_type, "MEDIUM POLISHED%")
        & ISIN(size, [49, 14, 23, 45, 19, 3, 36, 9])
    )
    .supply_records(
        p_brand=BACK(1).brand,
        p_type=BACK(1).part_type,
        p_size=BACK(1).size,
        ps_suppkey=supplier_key,
    )
    .WHERE(~LIKE(supplier.comment, "%Customer%Complaints%"))
)
output = PARTITION(selected_records, name="ps", by=(p_brand, p_type, p_size))(
    P_BRAND=p_brand,
    P_TYPE=p_type,
    P_SIZE=p_size,
    SUPPLIER_COUNT=NDISTINCT(ps.supplier_key),
).ORDER_BY(SUPPLIER_COUNT.DESC(), P_BRAND.ASC(), P_TYPE.ASC(), P_SIZE.ASC())
pydough.to_df(output)

## Query 17

This question seeks to determine **how much average yearly revenue would be lost if orders were no longer filled for small quantities of certain parts**.

Here is the corresponding SQL:

```SQL
select
    sum(l_extendedprice) / 7.0 as avg_yearly
from
    lineitem,
    part
where
    p_partkey = l_partkey
    and p_brand = 'Brand#23'
    and p_container = 'MED BOX'
    and l_quantity < (
        select
            0.2 * avg(l_quantity)
        from
            lineitem
        where
            l_partkey = p_partkey
    )
```

In [ ]:
%%pydough

selected_lines = parts.WHERE((brand == "Brand#23") & (container == "MED BOX"))(
    avg_quantity=AVG(lines.quantity)
).lines.WHERE(quantity < 0.2 * BACK(1).avg_quantity)
output = TPCH(AVG_YEARLY=SUM(selected_lines.extended_price) / 7.0)
pydough.to_df(output)

## Query 18

This question seeks to **ranks customers based on their status as having placed a large quantity order**.

Here is the corresponding SQL:

```SQL
select
    c_name,
    c_custkey,
    o_orderkey,
    o_orderdate,
    o_totalprice,
    sum(l_quantity)
from
    customer,
    orders,
    lineitem
where
    o_orderkey in (
        select
            l_orderkey
        from
            lineitem
        group by
            l_orderkey 
        having
            sum(l_quantity) > 300
        )
    and c_custkey = o_custkey
    and o_orderkey = l_orderkey
group by
    c_name,
    c_custkey,
    o_orderkey,
    o_orderdate,
    o_totalprice
order by
    o_totalprice desc,
    o_orderdate;
```

In [ ]:
%%pydough

output = orders(
    C_NAME=customer.name,
    C_CUSTKEY=customer.key,
    O_ORDERKEY=key,
    O_ORDERDATE=order_date,
    O_TOTALPRICE=total_price,
    TOTAL_QUANTITY=SUM(lines.quantity),
).WHERE(TOTAL_QUANTITY > 300).ORDER_BY(O_TOTALPRICE.DESC(), O_ORDERDATE.ASC())
pydough.to_df(output)

## Query 19

This question seeks to **report the gross discounted revenue resulting from select parts with a set of handling details**.

Here is the corresponding SQL:

```SQL
select
    sum(l_extendedprice * (1 - l_discount) ) as revenue
from
    lineitem,
    part
where
    (
        p_partkey = l_partkey
        and p_brand = 'Brand#12'
        and p_container in ( 'SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
        and l_quantity >= 1 and l_quantity <= 1 + 10
        and p_size between 1 and 5
        and l_shipmode in ('AIR', 'AIR REG')
        and l_shipinstruct = 'DELIVER IN PERSON'
    )
    or
    (
        p_partkey = l_partkey
        and p_brand = 'Brand#23'
        and p_container in ('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK')
        and l_quantity >= 10 and l_quantity <= 10 + 10
        and p_size between 1 and 10
        and l_shipmode in ('AIR', 'AIR REG')
        and l_shipinstruct = 'DELIVER IN PERSON'
    )
    or
    (
        p_partkey = l_partkey
        and p_brand = 'Brand#34'
        and p_container in ('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG')
        and l_quantity >= 20 and l_quantity <= 20 + 10
        and p_size between 1 and 15
        and l_shipmode in ('AIR', 'AIR REG')
        and l_shipinstruct = 'DELIVER IN PERSON'
    )
```

In [ ]:
%%pydough

selected_lines = lines.WHERE(
    (ISIN(ship_mode, ("AIR", "AIR REG")))
    & (ship_instruct == "DELIVER IN PERSON")
    & (part.size >= 1)
    & (
        (
            (part.size <= 5)
            & (quantity >= 1)
            & (quantity <= 11)
            & ISIN(
                part.container,
                ("SM CASE", "SM BOX", "SM PACK", "SM PKG"),
            )
            & (part.brand == "Brand#12")
        )
        | (
            (part.size <= 10)
            & (quantity >= 10)
            & (quantity <= 20)
            & ISIN(
                part.container,
                ("MED BAG", "MED BOX", "MED PACK", "MED PKG"),
            )
            & (part.brand == "Brand#23")
        )
        | (
            (part.size <= 15)
            & (quantity >= 20)
            & (quantity <= 30)
            & ISIN(
                part.container,
                ("LG CASE", "LG BOX", "LG PACK", "LG PKG"),
            )
            & (part.brand == "Brand#34")
        )
    )
)
output = TPCH(
    REVENUE=SUM(selected_lines.extended_price * (1 - selected_lines.discount))
)
pydough.to_df(output)

## Query 20

This question seeks to **determine supppliers in a particular nation that may be candidate for discounts over a year in question.**.

Here is the corresponding SQL:

```SQL
select
    s_name,
    s_address
from
    supplier, nation
where
    s_suppkey in (
        select
            ps_suppkey
        from
            partsupp
        where
            ps_partkey in (
                select
                    p_partkey
                from
                    part
                where
                    p_name like 'forest%'
            )
            and ps_availqty > (
                select
                    0.5 * sum(l_quantity)
                from
                    lineitem
                where
                    l_partkey = ps_partkey
                    and l_suppkey = ps_suppkey
                    and l_shipdate >= date('1994-01-01')
                    and l_shipdate < date('1994-01-01') + interval '1' year
            )
    )
    and s_nationkey = n_nationkey
    and n_name = 'CANADA'
order by
    s_name
```

Notice that the SQL version requires repeated use of subqueries, but PyDough can combine the `part_qty` and `selected_part_supplied` sections into a single extended expression that cleanly represents the filter requirement.

In [ ]:
%%pydough

part_qty = SUM(
    lines.WHERE(
        (ship_date >= datetime.date(1994, 1, 1))
        & (ship_date < datetime.date(1995, 1, 1))
    ).quantity
)
selected_part_supplied = supply_records.part.WHERE(
    STARTSWITH(name, "forest") & (BACK(1).availqty > part_qty * 0.5)
)
output = suppliers(
    S_NAME=name,
    S_ADDRESS=address,
).WHERE((nation.name == "CANADA") & COUNT(selected_part_supplied) > 0).ORDER_BY(S_NAME.ASC())
pydough.to_df(output)